# Solving systems of Linear Equations Using a Quantum Computer

# 1) Introduction

Systems of linear equations have a lot of applications in science. So if we find a speedup on solving this problem there is real impact on society. 

The problem that we want to solve is of the form:

$$
A x = b
$$

Where A is a matrix and b is given, we want to solve for x. Classicaly there is two types of solutions:

- 1) **LU Decomposition**: That finds x in $O(N^{2.376} \mathrm{poly}(log(k/\epsilon)))$ where $\epsilon$ is a bound of x error and k is the ith singular value, $k = ||A|| ||A^{-1}|| = \frac{\sigma_1(A)}{\sigma_N(A)} \sigma_1(A)$.

- 2) **Iterative Methods**: Needs $O(\sqrt{k} \ \mathrm{log}(1/\epsilon))$ matrix vector multiplications. If A is spase, then we need $O(Nd\sqrt{k} \ \mathrm{log}(1/\epsilon))$ where d is the number of non-zero entries per row.

# 2) Quantum Algorithm

## 2.1) Introduction
For the implementation of this algorithm is important to understand the Quantum Phase Estimation algorithm, because this is an application of this algorithm.

For the quantum algorithm we need to convert the linear equation problem for a quantum mechanical problem, to do this we represent b as quantum state: $$ | b \rangle = \sum_{i=1}^N b_i | i \rangle$$ and  the problem is: 

$$
A | x \rangle = | b \rangle
$$

we assume that A is an hermitian operator (Matrix), that means if A is not Hermitian we expand A in order to be hermitian:

$$
C = \begin{pmatrix} 0 & A \\ A^{\dagger} & 0 \end{pmatrix}
$$

And change the problem to be $$Cy = \begin{pmatrix} b \\ 0 \end{pmatrix} \ \ \ , \ \ y = \begin{pmatrix} 0 \\ x \end{pmatrix}$$

The algorithm is called HHL because of the three authors of the original [paper](https://arxiv.org/abs/0811.3171):  Harrow, Hassidim and Lloyd. 

## 2.2) Overview of the Algorithm

The HHL consists of 3 registers, which we denote as $A'$ for the ancilla, W for work register, and IO for Input/Output register. 

The input of the algorithm is the quantum state of b, $| b \rangle$, which is input to the IO register. All other registers starts on the $| 0 \rangle$, so we have:

$$
| \psi_0 \rangle \equiv |0 \rangle_{A'} \otimes | 0 \rangle_{W} \otimes | b \rangle_{IO}
$$

In addition, we also have the matrix A as the input of the algorithm. The algorithm works on three steps:

- 1) Quantum Phase Estimation with the unitary $U_A \equiv e^{iAt}$ controlled by the W register and $U_A$ applied to the IO register.

- 2) Pauli-Y rotation for a particular angle $\theta$ on the A' register controlled by the W register.

- 3) Do the first step in reverse, that is do the Inverse Qunatum Phase Estimation for $U_A$, on the W register.

If the register $A'$ is measured and one post-select (That is, we select only the state with the given outcome) on the $| 1 \rangle_{A'}$ outcome, then the state of the IO register will be close to $| x \rangle$.

## 2.3) Algorithm's Walkthrough

Since we assume that the matrix A is hermitian, then because of the spectral theoram A can be written in the eigenbasis:

$$
A = \sum_j \lambda_j | u_j \rangle \langle u_j |
$$

Also because A is hermitian, the operator $U_A$ is unitary and has eigenvalues $e^{i \lambda_j t}$ and eigenstates $| u_j \rangle$. Then we apply the QPE:

$$
| \psi_1 \rangle = | 0 \rangle_{A'} \otimes \sum_j \beta_j | \overline{\lambda_j} \rangle_{W} \otimes | u_j \rangle_{IO}
$$

Where $| \overline{\lambda_j} \rangle$ is a binary representation of $\lambda_j$ up to a set precision and $$\sum_j \beta_j | u_j \rangle_{IO}$$ is the expansion of $| b \rangle$ in the eigenbasis $| u_j \rangle$. 

Applying the second step, which is the controlled-Y rotation $e^{i \theta Y}$ for the angle:

$$
\theta = \mathrm{arccos} \frac{C}{\lambda_j}
$$

Where C is an hyperparameter of the algorithm and is set by the user. After this rotation on the A' register, we have:

$$
| \psi_2 \rangle = \sum_j \beta_j \bigg( \sqrt{ 1 - \frac{C^2}{\overline{\lambda_j}^2}} | 0 \rangle_{A'} + + \frac{C}{\overline{\lambda_j}} | 0 \rangle_{A'} \bigg)\otimes | \overline{\lambda_j} \rangle_{W} \otimes | u_j \rangle_{IO}
$$

The third step is to do de reverse QPE, this gives:

$$
| \psi_3 \rangle = \sum_j \beta_j \bigg( \sqrt{ 1 - \frac{C^2}{\overline{\lambda_j}^2}} | 0 \rangle_{A'} + + \frac{C}{\overline{\lambda_j}} | 0 \rangle_{A'} \bigg)\otimes | 0 \rangle_{W} \otimes | u_j \rangle_{IO}
$$

This state is already on the inverse form, because: $$ A^{-1} | b \rangle = \sum_{j=1}^N \frac{\beta_j}{\lambda_j} | u_j \rangle$$

Therefore if we measure the A' register and post-select $| 1 \rangle_{A'}$, we have the solution:

$$
| \psi_4 \rangle = \sum_{j=1}^N \frac{\beta_j}{\lambda_j} | u_j \rangle_{IO} \approx | x \rangle
$$

Thus the IO register contains the approximation of $|x \rangle$.

This solves the linear equation problem exponentially faster than a classical computer, but there's a caveat (as always), the solution vector is a quantum state, therefore for getting the classical description of x we need to do quantum state tomography (QST) but QST scales exponentially with the number of qubits loosing the exponential speedup. There are some applications that we only need an expectation value, this leads to an exponential speedup.

*Note*: A good value for the hyperparameter C is to be the smallest eigenvalue that can be represented by the circuit, that is 

$$
C = \frac{2 \pi}{2^n t}
$$

# 3) Qiskit implementation

For solving this problem we will use the qiskit.aqua library, that has implementations of more complex algorithms on qiskit.

The way that Aqua works is to put together components of the algorithm, for instance, we need to instantiate the QPE outside the HHL algorithm and put this as the input of the HHL algorithm. 

Another component that we need to instantiate is the LookupRotation that is the controlled-Y rotation on step 2 of the algorithm.

In [89]:
from qiskit import Aer
from qiskit.circuit.library import QFT
from qiskit.aqua import QuantumInstance, aqua_globals
from qiskit.quantum_info import state_fidelity
from qiskit.aqua.algorithms import HHL, NumPyLSsolver
from qiskit.aqua.components.eigs import EigsQPE
from qiskit.aqua.components.reciprocals import LookupRotation
from qiskit.aqua.operators import MatrixOperator
from qiskit.aqua.components.initial_states import Custom
import numpy as np

In [90]:
def create_eigs(matrix, num_ancillae, num_time_slices, negative_evals):
    """QPE for the eigenvalues estimation.

    Parameters
    -------------------------------------------------------------------
    matrix(np.array): Unitary matrix for the QPE.
    num_ancillae(int): Number of ancillas.
    num_time_slices(float): An optional evolution time which should scale the 
                            eigenvalue onto the range (0,1] (or (−0.5,0.5] for                                  
                            negative eigenvalues).If None is internally computed.
    negative_evals(Boolean): Set True to indicate negative eigenvalues need to be handled

    """
    
    ne_qfts = [None, None]
    
    if negative_evals:
        num_ancillae += 1
        ne_qfts = [QFT(num_ancillae - 1), QFT(num_ancillae - 1).inverse()]

    return EigsQPE(MatrixOperator(matrix=matrix),
                   QFT(num_ancillae).inverse(),
                   num_time_slices=num_time_slices,
                   num_ancillae=num_ancillae,
                   expansion_mode='suzuki',
                   expansion_order=2,
                   evo_time=None,
                   negative_evals=negative_evals,
                   ne_qfts=ne_qfts)

def fidelity(hhl, ref):
    """ Helper function for the fidelity comparing 
    the solution from the HHL and the classical solution.
    """

    solution_hhl_normed = hhl / np.linalg.norm(hhl)
    solution_ref_normed = ref / np.linalg.norm(ref)
    fidelity = state_fidelity(solution_hhl_normed, solution_ref_normed)        
    return fidelity    

## 3.1) Running on the statevector_simulation

Let's solve the HHL algorithm for the following problem:

$$
A = \begin{pmatrix} 1 & \frac{1}{3} \\ \frac{1}{3} & 1\end{pmatrix} \ \ \ \ \ b = \begin{pmatrix} 1 \\ 4 \end{pmatrix}
$$

In [91]:
A = [[1, 1/3], [1/3, 4]]
b = [1, 4]

In [92]:
orig_size = len(b)
A, b, truncate_powerdim, truncate_hermitian = HHL.matrix_resize(A, b)

# Initialize eigenvalue finding module
eigs = create_eigs(matrix=A, num_ancillae=3, 
                   num_time_slices=50, 
                   negative_evals=False)

num_q, num_a = eigs.get_register_sizes()

# Initialize initial state module
init_state = Custom(num_q, state_vector=b)

# Initialize reciprocal rotation module
reci = LookupRotation(negative_evals=eigs._negative_evals, 
                      evo_time=eigs._evo_time,
                      scale=0.5)

algo = HHL(matrix=A, vector=b, truncate_powerdim=truncate_powerdim, 
           truncate_hermitian=truncate_hermitian, eigs=eigs,
           init_state=init_state, reciprocal=reci, 
           num_q= num_q, num_a=num_a, orig_size=orig_size)

result = algo.run(QuantumInstance(Aer.get_backend('statevector_simulator')))

print(f"solution: {np.round(result['solution'], 5)}")

result_ref = NumPyLSsolver(A, b).run()
print(f"classical solution: {np.round(result_ref['solution'], 5)} ")

print(f"probability: {result['probability_result']}")

fidelity(result['solution'], result_ref['solution'])

solution: [0.31963-0.j 0.99125-0.j]
classical solution: [0.68571 0.94286] 
probability: 0.3961689326934409


0.9029076959066863

## 3.2) Using qasm_simulator

In [96]:
orig_size = len(b)
A, b, truncate_powerdim, truncate_hermitian = HHL.matrix_resize(A, b)

# Initialize eigenvalue finding module
eigs = create_eigs(matrix=A, num_ancillae=3, 
                   num_time_slices=50, 
                   negative_evals=False)

num_q, num_a = eigs.get_register_sizes()

# Initialize initial state module
init_state = Custom(num_q, state_vector=b)

# Initialize reciprocal rotation module
reci = LookupRotation(negative_evals=eigs._negative_evals, 
                      evo_time=eigs._evo_time,
                      scale=0.5)

algo = HHL(matrix=A, vector=b, truncate_powerdim=truncate_powerdim, 
           truncate_hermitian=truncate_hermitian, eigs=eigs,
           init_state=init_state, reciprocal=reci, 
           num_q= num_q, num_a=num_a, orig_size=orig_size)

result = algo.run(QuantumInstance(Aer.get_backend('qasm_simulator')))

print(f"solution: {np.round(result['solution'], 5)}")

result_ref = NumPyLSsolver(A, b).run()
print(f"classical solution: {np.round(result_ref['solution'], 5)} ")

print(f"probability: {result['probability_result']}")

fidelity(result['solution'], result_ref['solution'])

solution: [0.33   +0.j 0.98999+0.j]
classical solution: [0.68571 0.94286] 
probability: [0.41015625 0.43164062 0.41015625]


0.9086486486486484

References:

[1] - [Qiskit Legacy Tutorials](https://github.com/Qiskit/qiskit-tutorials/blob/master/legacy_tutorials/aqua/linear_systems_of_equations.ipynb);

[2] - [HHL Paper](https://arxiv.org/abs/0811.3171);

[3] - [J. Hidary - Quantum Computing: An Applied Approach](https://www.springer.com/gp/book/9783030239213);

[4] - [Qiskit Book](https://qiskit.org/textbook/ch-applications/hhl_tutorial.html).

-----------------------------------------------------------------

In [4]:
from qiskit.tools.jupyter import *
%qiskit_version_table